# Init

In [1]:
### Basic Imports ###
import builtins, csv, datetime, os, subprocess, time, types, pprint
from time import sleep
from pprint import pprint
now = time.time

### Special Imports ###
import numpy as np
from numpy import sqrt

import py_trees
from py_trees.behaviour import Behaviour
from py_trees.common import Status
from py_trees.composites import Selector, Sequence
from py_trees.decorators import FailureIsSuccess
import py_trees.console as console

### Local Imports ###
from magpie.poses import pose_error


### Restore Vars ###
# %store -r

# Behaviors

In [2]:
########## BASE CLASS ##############################################################################

class BasicBehavior( Behaviour ):
    """ Abstract class for repetitive housekeeping """
    
    
    def __init__( self, name = None, ctrl = None ):
        """ Set name to the child class name unless otherwise specified """
        if name is None:
            super().__init__( name = str( self.__class__.__name__  ) )
        else:
            super().__init__( name = name )
        self.ctrl = ctrl
        self.logger.debug( f"[{self.name}::__init__()]" )
        if self.ctrl is None:
            self.logger.warn( f"{self.name} is NOT conntected to a robot controller!" )

        
    def setup(self):
        self.logger.debug( f"[{self.name}::setup()]" )          
        
        
    def initialise( self ):
        """ Run first time behaviour is ticked or not RUNNING.  Will be run again after SUCCESS/FAILURE. """
        self.status = Status.RUNNING # Do not let the behavior idle in INVALID
        self.logger.debug( f"[{self.name}::initialise()]" )          

        
    def terminate( self, new_status ):
        """ Log how the behavior terminated """
        self.status = new_status
        self.logger.debug( f"[{self.name}::terminate().terminate()][{self.status}->{new_status}]" )
        
        
    def update( self ):
        """ Return true in all cases """
        self.status = py_trees.common.Status.SUCCESS
        return self.status

In [3]:
########## CONSTANTS & COMPONENTS ##################################################################

### Init data structs & Keys ###
builtins._DUMMYPOSE     = np.eye(4)
builtins.MP2BB = dict()  # Hack the BB object into the built-in namespace
builtins.SCAN_POSE_KEY  = "scanPoses"
builtins.HAND_OBJ_KEY   = "handHas"
PROTO_PICK_ROT = np.array( [[ 0.0,  1.0,  0.0, ],
                            [ 1.0,  0.0,  0.0, ],
                            [ 0.0,  0.0, -1.0, ]] )

### Set important BB items ###
MP2BB[ SCAN_POSE_KEY ] = dict()



In [4]:
########## MOVEMENT BEHAVIORS ######################################################################

### Constants ###
LIBBT_TS_S       = 0.25
DEFAULT_TRAN_ERR = 0.002
DEFAULT_ORNT_ERR = 3*np.pi/180.0

In [5]:
##### Move_Q #####################################


class Move_Q( BasicBehavior ):
    """ Move the joint config `qPos` """
    
    def __init__( self, qPos, name = None, ctrl = None, rotSpeed = 1.05, rotAccel = 1.4, asynch = True ):
        """ Set the target """
        # NOTE: Asynchronous motion is closest to the Behavior Tree paradigm, Avoid blocking!
        super().__init__( name, ctrl )
        self.qPos     = qPos
        self.rotSpeed = rotSpeed
        self.rotAccel = rotAccel
        self.asynch   = asynch
    
    
    def initialise( self ):
        """ Actually Move """
        super().initialise()
        self.ctrl.moveJ( self.qPos, self.rotSpeed, self.rotAccel, self.asynch )
    
    
    def update( self ):
        """ Return SUCCESS if the target reached """
        if self.ctrl.p_moving():
            self.status = Status.RUNNING
        else:
            error = np.subtract( self.qPos, self.ctrl.get_joint_angles() )
            error = error.dot( error )
            if( error > 0.1 ):
                self.status = Status.FAILURE
            else:
                self.status = Status.SUCCESS 
        return self.status

In [6]:
##### Move_Arm ###################################
    
    
class Move_Arm( BasicBehavior ):
    """ Move linearly in task space to the designated pose """
    
    def __init__( self, pose, name = None, ctrl = None, linSpeed = 0.25, linAccel = 0.5, asynch = True ):
        """ Set the target """
        # NOTE: Asynchronous motion is closest to the Behavior Tree paradigm, Avoid blocking!
        super().__init__( name, ctrl )
        self.pose     = pose
        self.linSpeed = linSpeed
        self.linAccel = linAccel
        self.asynch   = asynch
        
        
    def initialise( self ):
        """ Actually Move """
        super().initialise()
        self.ctrl.moveL( self.pose, self.linSpeed, self.linAccel, self.asynch )
        
        
    def update( self ):
        """ Return true if the target reached """
        if self.ctrl.p_moving():
            self.status = Status.RUNNING
        else:
            pM = self.ctrl.get_tcp_pose()
            pD = self.pose
            [errT, errO] = pose_error( pM, pD )
            if (errT <= DEFAULT_TRAN_ERR) and (errO <= DEFAULT_ORNT_ERR):
                self.status = Status.SUCCESS
            else:
                print( self.name, ", POSE ERROR:", [errT, errO] )
                self.status = Status.FAILURE
        return self.status

In [7]:
##### Open_Hand ##################################
    
    
class Open_Gripper( BasicBehavior ):
    """ Open fingers to max extent """
    
    def __init__( self, name = None, ctrl = None  ):
        """ Set the target """
        super().__init__( name, ctrl )
        self.wait_s = 0.5
        
        
    def initialise( self ):
        """ Actually Move """
        super().initialise()
        self.ctrl.open_gripper()
        sleep( self.wait_s )
        
        
    def update( self ):
        """ Return true if the target reached """
        self.status = Status.SUCCESS
        return self.status
        

In [8]:
##### Set_Fingers ##################################
    
    
class Set_Gripper( BasicBehavior ):
    """ Open fingers to max extent """
    
    def __init__( self, width_m, name = None, ctrl = None  ):
        """ Set the target """
        super().__init__( name, ctrl )
        self.width_m = width_m
        self.wait_s = 0.5
        
        
    def initialise( self ):
        """ Actually Move """
        super().initialise()
        self.ctrl.set_gripper( self.width_m )
        sleep( self.wait_s )
        
    
    def update( self ):
        """ Return true if the target reached """
        self.status = Status.SUCCESS
        return self.status

In [9]:
##### Close_Hand ##################################
    
    
class Close_Gripper( BasicBehavior ):
    """ Close fingers completely """
    
    def __init__( self, name = None, ctrl = None  ):
        """ Set the target """
        super().__init__( name, ctrl )
        self.wait_s = 0.5
        
        
    def initialise( self ):
        """ Actually Move """
        super().initialise()
        self.ctrl.close_gripper()
        sleep( self.wait_s )
        
        
    def update( self ):
        """ Return true if the target reached """
        self.status = Status.SUCCESS
        return self.status

In [10]:
##### Jog_Safe ###################################

class Jog_Safe( Sequence ):
    """ Move to a target by traversing at a safe altitude """
    # NOTE: This behavior should not, on its own, assume any gripper state
    
    def __init__( self, endPose, zSAFE=0.150, name="Jog_Safe", 
                  ctrl  = None ):
        """Construct the subtree"""
        super().__init__( name = name, memory = True )
        
        # Init #
        self.zSAFE = max( zSAFE, endPose[2,3] ) # Eliminate (some) silly vertical movements
        self.ctrl  = ctrl
        
        # Poses to be Modified at Ticktime #
        self.targetP = endPose.copy()
        self.pose1up = _DUMMYPOSE.copy()
        self.pose2up = _DUMMYPOSE.copy()
        
        # Behaviors whose poses will be modified #
        self.moveUp = Move_Arm( self.pose1up, ctrl=ctrl )
        self.moveJg = Move_Arm( self.pose2up, ctrl=ctrl )
        self.mvTrgt = Move_Arm( self.targetP, ctrl=ctrl )
        
        
        # 1. Move direcly up from the starting pose
        self.add_child( self.moveUp )
        # 2. Translate to above the target
        self.add_child( self.moveJg )
        # 3. Move to the target pose
        self.add_child( self.mvTrgt )
       
        
    def initialise( self ):
        """
        ( Ticked first time ) or ( ticked not RUNNING ):
        Generate move waypoint, then move with condition
        """
        nowPose = self.ctrl.get_tcp_pose()
        
        self.pose1up = nowPose.copy()
        self.pose1up[2, 3] = self.zSAFE

        self.pose2up = self.targetP.copy()
        self.pose2up[2, 3] = self.zSAFE

        self.moveUp.pose = self.pose1up.copy()
        self.moveJg.pose = self.pose2up.copy()
        self.mvTrgt.pose = self.targetP.copy()

In [11]:
########## MANIPULATION BEHAVIORS ##################################################################


class Pick_at_Pose( Sequence ):
    """ Grasp at a target pose (Robot Frame) while traversing at a safe altitude """
    # NOTE: This behavior should not, on its own, assume any gripper state

    def __init__( self, target, zSAFE = 0.150, preGraspW_m = None, name = "Pick_at_Pose", ctrl = None ):
        """Construct the subtree"""
        super().__init__( name = name, memory = True)
        self.ctrl = ctrl
        
        # 1. Open the gripper
        if preGraspW_m is None:
            self.add_child(  Open_Gripper( name = "Open", ctrl = ctrl )  )
        else:
            self.add_child(  Set_Gripper( preGraspW_m, name = "Open", ctrl = ctrl )  )
        # 2. Jog to the target
        self.add_child(  Jog_Safe( target, name = "Jog to Grasp Pose", ctrl = ctrl )  )
        # 1. Close the gripper
        self.add_child(  Close_Gripper( name = "Close", ctrl = ctrl )  )

In [12]:
########## EXECUTION ###############################################################################


class HeartRate: 
    """ Sleeps for a time such that the period between calls to sleep results in a frequency not greater than the specified 'Hz' """
    # NOTE: This fulfills a purpose similar to the rospy rate
    
    def __init__( self , Hz ):
        """ Create a rate object with a Do-Not-Exceed frequency in 'Hz' """
        self.period = 1.0 / Hz; # Set the period as the inverse of the frequency , hearbeat will not exceed 'Hz' , but can be lower
        self.last = time.time()
        
    def check_elapsed( self, reset = True ):
        """ Check if the period has elapsed, Optionally `reset` the clock """
        elapsed = time.time() - self.last
        update  = elapsed >= self.period
        if( update and reset ):
            self.last = time.time()
        return update
    
    def sleep( self ):
        """ Sleep for a time so that the frequency is not exceeded """
        elapsed = time.time() - self.last
        if elapsed < self.period:
            time.sleep( self.period - elapsed )
        self.last = time.time()


""" Return a formatted timestamp string, useful for logging and debugging """
def nowTimeStamp(): return datetime.datetime.now().strftime(
    '%Y-%m-%d_%H-%M-%S')  # http://stackoverflow.com/a/5215012/893511



class StopBeetle:
    """Invasive Beetle: Kills (stops) all branches of the tree"""

    def __init__(self, killStatus):
        """Set the status that will be assigned to all branches"""
        self.status = killStatus

    def run(self, behav):
        """Kill all subtrees"""
        for chld in behav.children:
            self.run(chld)
        behav.stop(self.status)


        
def run_BT_until_done(
    rootNode,
    N              = 10000,
    tickPause      =     0.25,
    Nverb          =    50,
    breakOnFailure = True,
    breakOnSuccess = True,
    treeUpdate     = 0,
    failureTree    = 1,
    successTree    = 0,
):
    """Tick root until `maxIter` is reached while printing to terminal"""

    if Nverb:
        print(
            "About to run",
            type( rootNode ),
            "named",
            rootNode.name,
            "at",
            nowTimeStamp(),
            "with",
            1 / tickPause,
            "Hz update frequency ...",
        )

    # 0. Setup
    # rootNode.setup_subtree( childrenFirst = 0 )
    rootNode.setup_with_descendants()
    pacer = HeartRate(Hz=1 / tickPause)  # metronome

    if Nverb:
        print("Running ...\n")

    # 1. Run
    for i in range(1, N + 1):
        try:
            rootNode.tick_once()

            if Nverb > 0 and i % Nverb == 0:
                print("\n--------- Tick {0} ---------\n".format(i))
                print("Root node, Name:", rootNode.name, ", Status:", rootNode.status)
                print("\n")
                if treeUpdate:
                    print(
                        py_trees.display.unicode_tree(root=rootNode, show_status=True)
                    )

            if breakOnFailure and (rootNode.status == Status.FAILURE):
                print("Root node", rootNode.name, "failed!\n")
                if failureTree:
                    print(
                        py_trees.display.unicode_tree(root=rootNode, show_status=True)
                    )
                break
            elif breakOnSuccess and (rootNode.status == Status.SUCCESS):
                print("Root node", rootNode.name, "succeeded!\n")
                if successTree:
                    print(
                        py_trees.display.unicode_tree(root=rootNode, show_status=True)
                    )
                break
            else:
                pacer.sleep()

        except KeyboardInterrupt:
            print("\nSimulation HALTED by user at", nowTimeStamp())
            break

    print("\nRun completed! with status:", rootNode.status, "\n\n")

    insect = StopBeetle(rootNode.status)

    for i in range(3):
        rootNode.visit(insect)  # HACK required coz tree doesn't complete sometimes
        sleep(0.5)

    print("Root node", rootNode.name, "was killed by the running script!")

# Test

In [13]:
from magpie.ur5 import UR5_Interface

In [14]:
robot = UR5_Interface()

In [15]:
robot.start()

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 


In [51]:
b1 = Set_Gripper(0.08, name = 'set gripper', ctrl =  robot)
b2 = Open_Gripper(name = 'open gripper', ctrl = robot)

In [52]:
run_BT_until_done(b1)

About to run <class '__main__.Set_Gripper'> named set gripper at 2024-04-26_14-47-19 with 4.0 Hz update frequency ...
Running ...

Position of dxl ID: 1 set to 389 
Position of dxl ID: 2 set to 650 
Root node set gripper succeeded!


Run completed! with status: Status.SUCCESS 


Root node set gripper was killed by the running script!


In [18]:
run_BT_until_done(b2)

About to run <class '__main__.Open_Gripper'> named open gripper at 2024-04-26_14-36-11 with 4.0 Hz update frequency ...
Running ...

Position of dxl ID: 1 set to 303 
Position of dxl ID: 2 set to 729 
Root node open gripper succeeded!


Run completed! with status: Status.SUCCESS 


Root node open gripper was killed by the running script!


In [53]:
##### Get Gripper Force ###################
    
class Get_Gripper_Force( BasicBehavior ):
    def __init__( self, name = None, ctrl = None  ):
        """ Set the target """
        super().__init__( name, ctrl )
        self.wait_s = 0.5
        
        
    def initialise( self ):
        """ Actually get the force """
        super().initialise()
        print(self.ctrl.get_gripper_force())
        sleep( self.wait_s )

In [63]:
bF = Get_Gripper_Force( name = 'get gripper force', ctrl = robot )
run_BT_until_done(bF)

About to run <class '__main__.Get_Gripper_Force'> named get gripper force at 2024-04-26_14-49-07 with 4.0 Hz update frequency ...
Running ...

[0.1595425, 0.0]
Root node get gripper force succeeded!


Run completed! with status: Status.SUCCESS 


Root node get gripper force was killed by the running script!


In [21]:
##### Get Gripper Force ###################
    
class Get_Gripper_Voltage( BasicBehavior ):
    def __init__( self, name = None, ctrl = None  ):
        """ Set the target """
        super().__init__( name, ctrl )
        self.wait_s = 0.5
        
        
    def initialise( self ):
        """ Actually get the force """
        super().initialise()
        print(self.ctrl.get_gripper_voltage())
        sleep( self.wait_s )

In [31]:
bF = Get_Gripper_Voltage( name = 'get gripper voltage', ctrl = robot )
run_BT_until_done(bF)

About to run <class '__main__.Get_Gripper_Voltage'> named get gripper voltage at 2024-04-26_14-43-20 with 4.0 Hz update frequency ...
Running ...

[12.0, 12.1]
Root node get gripper voltage succeeded!


Run completed! with status: Status.SUCCESS 


Root node get gripper voltage was killed by the running script!


In [26]:
##### Get Gripper Force ###################
    
class Set_Gripper_Force( BasicBehavior ):
    def __init__( self, name = None, ctrl = None, force = 200 ):
        """ Set the target """
        super().__init__( name, ctrl )
        self.wait_s = 0.5
        self.force = force
        
        
    def initialise( self ):
        """ Actually get the force """
        super().initialise()
        self.ctrl.set_gripper_force(self.force)
        sleep( self.wait_s )

In [61]:
b = Set_Gripper_Force( name = 'get gripper force', ctrl = robot, force = 200)
run_BT_until_done(b)

About to run <class '__main__.Set_Gripper_Force'> named get gripper force at 2024-04-26_14-48-52 with 4.0 Hz update frequency ...
Running ...

Root node get gripper force succeeded!


Run completed! with status: Status.SUCCESS 


Root node get gripper force was killed by the running script!


In [ ]:
class Set_Gripper_Angles( BasicBehavior ):
    """ Open fingers to max extent """
    
    def __init__( self, angle1, angle2, name = None, ctrl = None  ):
        """ Set the target """
        super().__init__( name, ctrl )
        self.angle1 = angle1
        self.angle2 = angle2
        self.wait_s = 0.5
        
        
    def initialise( self ):
        """ Actually Move """
        super().initialise()
        self.ctrl.set_gripper_angles( self.angle1, self.angle2 )
        sleep( self.wait_s )
        
    
    def update( self ):
        """ Return true if the target reached """
        self.status = Status.SUCCESS
        return self.status

In [ ]:
# bx = Set_Gripper_Angles(100, 216, name = 'set gripper angles', ctrl =  robot)
# bx = Open_Gripper(name = 'open gripper', ctrl = robot)

In [ ]:
# run_BT_until_done(bx)

In [ ]:
from magpie import poses
# mtrx = poses.pose_vec_to_mtrx([-0.13138038, -0.59637503, 0.10047517, 2, 1, 1])

# print(mtrx)

# poseA = robot.get_tcp_pose()
# print(poseA)

# poseB = poses.translate_pose(poseA, [0, 0, 0], dir_pose="self")
# print(poseB)

In [ ]:
# b3 = Move_Arm(poseB, name = 'move arm', ctrl = robot, linSpeed = 0.25, linAccel = 0.5, asynch = True)

In [ ]:
# run_BT_until_done(b3)

In [ ]:
# bx = Set_Gripper_Angles(120, 216, name = 'set gripper angles', ctrl =  robot)
# b2 = Open_Gripper(name = 'open gripper', ctrl = robot)

In [ ]:
# run_BT_until_done(bx)

In [ ]:
# bx = Set_Gripper_Angles(140, 216, name = 'set gripper angles', ctrl =  robot)
# run_BT_until_done(bx)

In [ ]:
bx = Open_Gripper(name = 'open gripper', ctrl =  robot)
run_BT_until_done(bx)

[[ 0.99993051  0.00710841  0.00940482 -0.10754238]
 [ 0.00986928 -0.06840585 -0.99760876 -0.58571257]
 [-0.00644806  0.99763225 -0.06847125  0.2081723 ]
 [ 0.          0.          0.          1.        ]]

In [ ]:
# pick_behav = Pick_at_Pose(poseA, zSAFE = 0.150, preGraspW_m = None, name = "Pick_at_Pose", ctrl = robot)

In [ ]:
# run_BT_until_done(pick_behav)

In [ ]:
poseC = robot.get_joint_angles()
print(poseC)

# b4 = Move_Q(poseC, name = None, ctrl = robot, rotSpeed = 1.05, rotAccel = 1.4, asynch = True)

In [ ]:
# poseC[1] += 0.3
# print(poseC)

# b4 = Move_Q(poseC, name = None, ctrl = robot, rotSpeed = 1.05, rotAccel = 1.4, asynch = True)

In [ ]:
# run_BT_until_done(b4)

In [ ]:
pose_ = robot.get_tcp_pose()
print(pose_)

In [ ]:
# b5 = Pick_at_Pose(pose_, zSAFE = 0.150, preGraspW_m = None, name = "Pick_at_Pose", ctrl = robot)

In [ ]:
class Set_Gripper_Angles_After_Delay( BasicBehavior ):
    """ Set gripper angles after a time delay """
    
    def __init__( self, angle1, angle2, name = None, ctrl = None  ):
        """ Set the target """
        super().__init__( name, ctrl )
        self.angle1 = angle1
        self.angle2 = angle2
        self.wait_s = 0.5
        
        
    def initialise( self ):
        """ Actually Move """
        super().initialise()

        sleep( self.wait_s )
        self.ctrl.set_gripper_angles( self.angle1, self.angle2 )
        sleep( self.wait_s )
        
    
    def update( self ):
        """ Return true if the target reached """
        self.status = Status.SUCCESS
        return self.status

In [ ]:
##### Tip_Over_Box ###################################

from magpie import poses
    
class Tip_Over_Box( Sequence ):
    """ Tip over and pick up the box, assuming the correct start position. """

    def __init__( self, posesX, name = "Tip_Over_Box", ctrl = None ):
        """Construct the subtree"""
        super().__init__( name = name, memory = True)
        self.ctrl = ctrl
        self.posesX = posesX

        self.mvTrgt = Open_Gripper(ctrl=ctrl )
        self.add_child( self.mvTrgt )

        # POSITION GRIPPER TO TOP OF BOX
        self.mvTrgt = Move_Arm( self.posesX[0], ctrl=ctrl )
        self.add_child( self.mvTrgt )
        
        # MOVE GRIPPER TO CORRECT ORIENTATION
        self.mvTrgt = Move_Arm( self.posesX[1], ctrl=ctrl )
        self.add_child( self.mvTrgt )
        
        # MOVE TOP GRIPPER INTO POSITION
        self.mvTrgt = Set_Gripper_Angles( 120, 216, ctrl=ctrl )
        self.add_child( self.mvTrgt )

        # MOVE ARM DOWNWARD UNTIL CONTACT
        self.mvTrgt = Move_Arm( self.posesX[2], ctrl=ctrl )
        self.add_child( self.mvTrgt )

        # ADJUST GRIPPER POSITION
        self.mvTrgt = Set_Gripper_Angles( 120, 195, ctrl=ctrl )
        self.add_child( self.mvTrgt )

        # self.mvTrgt = Move_Arm( self.poses[5], ctrl=ctrl )
        # self.add_child( self.mvTrgt )

        # MOVE BACKWARD TO TIP BOX
        self.mvTrgt = Move_Arm( self.posesX[3], ctrl=ctrl )
        self.add_child( self.mvTrgt )

        # OPEN TOP GRIPPER TO ALLOW BOX TO FALL
        self.mvTrgt = Set_Gripper_Angles( 85, 195, ctrl=ctrl )
        self.add_child( self.mvTrgt )

        # # OPEN BOTTOM GRIPPER TO ADJUST BOX FURTHER
        # self.mvTrgt = Set_Gripper_Angles_After_Delay( 85, 216, ctrl=ctrl )
        # self.add_child( self.mvTrgt )

        # newPose = poses.translate_pose(np.array(posesX[3]), [-0.03, 0, 0.03], dir_pose="self")

        # # MOVE FORWARD
        # self.mvTrgt = Move_Arm( newPose, ctrl=ctrl )
        # self.add_child( self.mvTrgt )

        # # CLOSE TOP GRIPPER TO GRAB BOX
        # self.mvTrgt = Set_Gripper_Angles_After_Delay( 140, 216, ctrl=ctrl )
        # self.add_child( self.mvTrgt )

        # self.mvTrgt = Set_Gripper( 0.056, ctrl=ctrl )
        # self.add_child( self.mvTrgt )

        # self.mvTrgt = Move_Arm( self.posesX[4], ctrl=ctrl )
        # self.add_child( self.mvTrgt )

In [ ]:
posesX = [[[ 0.99789511,  0.0122757,  -0.06367615, -0.11642388],
 [-0.06399342,  0.02746575, -0.99757229, -0.67249229],
 [-0.01049699,  0.99954737,  0.0281935,   0.22991059],
 [ 0.,          0.,          0.,          1.        ]],
 [[ 0.00977942, -0.999209,   -0.03854528, -0.1163872 ],
 [-0.5803814,   0.02571804, -0.81393858, -0.67247468],
 [ 0.81428606,  0.03033081, -0.57967081,  0.30402206],
 [ 0.,          0.,          0.,          1.,        ]],
 [[ 0.0097527,  -0.99920752, -0.03859033, -0.11711042],
 [-0.58035142,  0.02577135, -0.81395827, -0.67245884],
 [ 0.81430775,  0.03033424, -0.57964017,  0.17557186],
 [ 0.,          0.,          0.,          1.        ]],
 [[ 0.01682541, -0.99877234, -0.04659097, -0.11879721],
 [-0.57637935,  0.02838736, -0.81668905, -0.59182293],
 [ 0.81700903,  0.0405952,  -0.57519412,  0.15571098],
 [ 0.,          0.,          0.,          1.,        ]],
 [[ 0.01684079, -0.99877571, -0.046513,   -0.11875802],
 [-0.57646161,  0.02831067, -0.81663365, -0.4122375 ],
 [ 0.81695067,  0.04056571, -0.57527909,  0.31449564],
 [ 0.,          0.,          0.,          1.,        ]]]

In [ ]:
# print(poses.pose_mtrx_to_vec( np.array([[ 0.01684079, -0.99877571, -0.046513,   -0.11875802],
#  [-0.57646161,  0.02831067, -0.81663365, -0.4122375 ],
#  [ 0.81695067,  0.04056571, -0.57527909,  0.31449564],
#  [ 0.,          0.,          0.,          1.,        ]])))

# pose_ = [[ 0.01684079, -0.99877571, -0.046513,   -0.11875802],
#  [-0.57646161,  0.02831067, -0.81663365, -0.4122375 ],
#  [ 0.81695067,  0.04056571, -0.57527909,  0.31449564],
#  [ 0.,          0.,          0.,          1.,        ]]
# # pose_ = poses.pose_vec_to_mtrx(np.array([-0.1, -0.4, 0.3, 0, np.pi / 4, 0]))
# # print(pose_)

# # alpha = 0
# # beta = -np.pi / 2.0
# # gamma = 0
# # pose_ = [[np.cos(alpha) * np.cos(beta)]]

# pose_ = [[ 1,  0,  0, -0.2],
#          [ 0,  0, -1, -0.8],
#          [ 0,  1,  0,  0.2],
#          [ 0., 0., 0., 1. ]]

# pose_ =  [[ 0.01682541, -0.99877234, -0.04659097, -0.11879721],
#  [-0.57637935,  0.02838736, -0.81668905, -0.59182293],
#  [ 0.81700903,  0.0405952,  -0.57519412,  0.15571098],
#  [ 0.,          0.,          0.,          1.,        ]]

# print(poses.pose_mtrx_to_vec(np.array(pose_)))
# pose_ = poses.pose_vec_to_mtrx(np.array([-0.1, -0.67, 0.30, np.pi / 2.0, -np.pi / 2.0, np.pi / 4.0]))



# x_coordinate = -0.2
# y_coordinate = -0.8
# rotation_amount = 0.0

# pose_ = poses.pose_vec_to_mtrx(np.array([x_coordinate, y_coordinate, 0.16, np.pi / 2.0, rotation_amount, rotation_amount]))
# b_ = Move_Arm(pose_, name = 'move arm', ctrl = robot, linSpeed = 0.25, linAccel = 0.5, asynch = True)
# run_BT_until_done(b_)

# pose_ = robot.get_tcp_pose()
# print(pose_)
# print(poses.pose_mtrx_to_vec(np.array(pose_)))




# poseC = robot.get_joint_angles()
# print(poseC)

# poseC = np.array([-np.pi / 2.0, -np.pi / 3.0, -np.pi / 2.0, -np.pi * 4.0 / 3.0, -np.pi / 2.0, np.pi / 2.0])
poseC = np.array([-np.pi / 2.0, -np.pi / 3.0, -np.pi / 2.0, -np.pi * 4.0 / 3.0, -np.pi / 2.0, 0])

b4 = Move_Q(poseC, name = None, ctrl = robot, rotSpeed = 1.05, rotAccel = 1.4, asynch = True)
run_BT_until_done(b4)

# print(poses.pose_vec_to_mtrx(poseC))

In [ ]:
robot.stop()

In [ ]:
bTB = Tip_Over_Box(posesX, name = 'Tipping Over Box', ctrl = robot)
run_BT_until_done(bTB)

In [ ]:
# poseZ = [[ 0.00977942, -0.999209,   -0.03854528, -0.1163872 ],
# [-0.5803814,   0.02571804, -0.81393858, -0.67247468],
# [ 0.81428606,  0.03033081, -0.57967081,  0.30402206],
# [ 0.,          0.,          0.,          1.        ]]

# startPose = [[1, 0, 0, -0.1],
#              [0, 0, -1, -0.6],
#              [0, 1, 0, 0.15],
#              [0, 0, 0, 1]]

# bZ = Move_Arm(startPose, name = 'move arm', ctrl = robot, linSpeed = 0.25, linAccel = 0.5, asynch = True)
# run_BT_until_done(bZ)

In [ ]:
from RealSense import RealSense
# import ObjectDetection as ob
# from magpie.ur5 import UR5_Interface
import open3d as o3d

In [ ]:
def displayWorld(worldPCD):
    coordFrame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.05)
    geometry = [coordFrame]
    geometry.append(worldPCD)
    o3d.visualization.draw_geometries(geometry)

In [ ]:
# urPose = robot.get_tcp_pose()
# print(urPose)

# urJointAngles = robot.get_joint_angles()
# print(urJointAngles)

# try:
#     detector = ob.ObjectDetection(real,None,moveRelative = True)
# except Exception as e:
#     # detector.real.pipe.stop()
#     raise(e)

real = RealSense(1.0)
real.initConnection()

pcd,rgbdImage = real.getPCD()
depthImage,colorImage = rgbdImage.depth,rgbdImage.color
# displayWorld(pcd)
# real.disconnect()

In [ ]:
real.displayImages(depthImage, colorImage)

In [ ]:
# real.displayPCD([pcd])

In [ ]:
# robot.stop()

In [ ]:
# poseA = [[ 0.99826425,  0.04047501, -0.04278145, -0.13138038],
#          [-0.02754608, -0.32117340, -0.94661970, -0.59637503],
#          [-0.05205471,  0.94615507, -0.31950100,  0.10047517],
#          [ 0.        ,  0.        ,  0.        ,  1.        ]]

# # open_gripper( self )

# __init__( self, pose, name = None, ctrl = None, linSpeed = 0.25, linAccel = 0.5, asynch = True ):

# b1 = Move_Arm( poseA, "Move to A", ctrl = robot )

# Set_Gripper( BasicBehavior ):
    
#     def __init__( self, width_m, name = None, ctrl = None  ):

In [ ]:
# b2 = Move_Arm( poseB, "Move to B", ctrl = robot )

In [ ]:
# run_BT_until_done( b2 )

In [ ]:
# s1 = Sequence( "Jog Jog", True )
# s1.add_children([
#     Jog_Safe( poseB, zSAFE = 0.300, name = "Jog B", ctrl = robot ),
#     Jog_Safe( poseA, zSAFE = 0.300, name = "Jog A", ctrl = robot ),
# ])

In [ ]:
# run_BT_until_done( s1 )

In [ ]:
# robot.stop()

# Setup

In [ ]:
# poseA = robot.get_tcp_pose()
# %store poseA

In [ ]:
# poseB = robot.get_tcp_pose()
# %store poseB